In [1]:
import os
import path 
import numpy as np
import pandas as pd
import re
import torch
import collections
import random

import scipy
import scipy.stats
import matplotlib.pyplot as plt
directory = os.getcwd()
folders = os.listdir(directory)

In [2]:
genomes = [name for name in os.listdir("/Users/tonytu/Desktop/all_post_processed_data/")]
genomes.remove('.DS_Store')
sorted_genomes = sorted(genomes)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/tonytu/Desktop/all_post_processed_data/'

In [3]:
pwd

'/Users/tonytu/Desktop/old_stuff/berkeley/fall2022/amiralilab'

In [5]:
len(sorted_genomes)

4288

# load data

We load all coding regions and non-coding regions and filter out the duplicates

In [6]:
random.seed(5)
torch.manual_seed(5)
train_x = np.array([])
train_y = np.array([])
test_x = np.array([])
test_y = np.array([])

def flatten(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list


def read_fasta(dir_name, genome_name, file_name):
    #'sampled_genomes_resampled/'
    f=open(str(dir_name) + '/' + str(genome_name) + '/' + file_name,'r')
    lines=f.readlines()

    hre=re.compile('>(\S+)')
    lre=re.compile('^(\S+)$')

    gene={}

    for line in lines:
            outh = hre.search(line)
            if outh:
                    id=outh.group(1)
            else:
                    outl=lre.search(line)
                    if(id in gene.keys()):
                            gene[id] += outl.group(1)
                    else:
                            gene[id]  =outl.group(1)
    return list(gene.values())

def create_csv(dir_name, genome, coding_seqs, sampled_noncoding_seqs):
    num_coding_seqs = len(coding_seqs)
    num_noncoding_seqs = len(sampled_noncoding_seqs)
    coding_indices = np.arange(0, num_coding_seqs, 1)
    coding_groundtruth = np.arange(0, num_coding_seqs, 1)
    coding_train = ['train'] * num_coding_seqs
    noncoding_indices = np.arange(0, num_noncoding_seqs, 1)
    noncoding_test = ['test'] * num_noncoding_seqs
    noncoding_groundtruth = np.arange(0, num_noncoding_seqs, 1)
    
    data = {'sequence': coding_seqs, 'target': coding_groundtruth, 'set': coding_train, 'validation': [np.nan] * num_coding_seqs}
    coding_df = pd.DataFrame(data, index=coding_indices)
    coding_df.to_csv(dir_name + '/' + genome + '/debug_coding_train.csv')
    
    data = {'sequence': sampled_noncoding_seqs, 'target': noncoding_groundtruth, 'set': noncoding_test, 'validation': [np.nan] * num_noncoding_seqs}
    noncoding_df = pd.DataFrame(data, index=noncoding_indices)
    noncoding_df.to_csv(dir_name + '/' + genome + '/debug_noncoding_test.csv')
    
def find_first_index(lst, target):
    """
    Finds the index of the first matching element in the given list.
    If no match is found, returns -1.
    """
    for i, item in enumerate(lst):
        if item == target:
            return i
    return -1
    
def create_dict(lst):
    result = {element: [] for element in lst}
    return result

def filter_duplicate_subdirectories(directories):
    """
    Returns a dictionary where the keys are directory names and the values are lists of subdirectories,
    filtered to exclude duplicates except for the first encounter of each subdirectory name
    in the order of directories sorted in lexicographic order.
    """
    filtered_directory_indices = create_dict(directories.keys())
    filtered_directory_genes = create_dict(directories.keys())
    subdirectory_set = set()
    for directory_name in sorted(directories.keys()):
        for i in range(len(directories[directory_name])):
            if not directories[directory_name][i] in subdirectory_set:
                filtered_directory_indices[directory_name].append(i)
                filtered_directory_genes[directory_name].append(directories[directory_name][i])
                subdirectory_set.add(directories[directory_name][i])
            else:
                continue
    return filtered_directory_genes

    

In [112]:
a = os.listdir('/Users/tonytu/Desktop/old_stuff/berkeley/fall2022/amiralilab/FragGeneScanRs-main/FLIP-main_latest/baselines/embeddings/gb1/esm1b/two_vs_rest/post_processed_data')
a.remove('.DS_Store')
len(a)


4288

# loading data for training

In [71]:
random.seed(5)
torch.manual_seed(5)
coding_genes = []
noncoding_genes = []
positive_genomes = {}
negative_genomes = {}

#note that all the coding regions are from the previous data directory (root_dir)
#all the noncoding regions are from the new data directory: all_post_processed_data
coding_regions_data_dir_path = root_dir
noncoding_regions_data_dir_path = '/Users/tonytu/Desktop/all_post_processed_data_old/'

for genome in genomes:
    # adding a length constraint
    length_upper_limit = 60
    length_lower_limit = 0
    positive_testing_sequences = list(pd.read_csv(coding_regions_data_dir_path + genome + '/coding_train.csv')['sequence'])
    negative_testing_sequences = list(pd.read_csv(noncoding_regions_data_dir_path + genome + '/all_noncoding_test.csv')['sequence'])
    positive_testing_sequences = [positive_testing_sequence for positive_testing_sequence in positive_testing_sequences if len(positive_testing_sequence) < 60]
    negative_testing_sequences = [negative_testing_sequence for negative_testing_sequence in negative_testing_sequences if len(negative_testing_sequence) < 60]
    coding_genes.extend(positive_testing_sequences)
    noncoding_genes.extend(negative_testing_sequences)
    
    positive_genomes[genome] = positive_testing_sequences
    negative_genomes[genome] = negative_testing_sequences
    

In [83]:
print('In total, there are ' + str(len(coding_genes)) + ' coding regions, out of which ' + str(len(set(coding_genes))) + ' are unique.')
print('In total, there are ' + str(len(noncoding_genes)) + ' noncoding regions, out of which ' + str(len(set(noncoding_genes))) + ' are unique.')

#all noncoding_regions are already unique

In total, there are 145232 coding regions, out of which 128615 are unique.
In total, there are 3367378 noncoding regions, out of which 3367378 are unique.


In [84]:
import collections
result = collections.Counter(coding_genes) & collections.Counter(noncoding_genes)
intersected_list = list(result.elements())
print(len(intersected_list))
print(len(set(intersected_list)))

1
1


In [85]:
#removing these from all datasets
unique_intersected_list = set(intersected_list)
for genome in genomes:
    for gene in positive_genomes[genome]:
        if gene in unique_intersected_list:
            positive_genomes[genome].remove(gene)
    for gene in negative_genomes[genome]:
        if gene in unique_intersected_list:
            negative_genomes[genome].remove(gene)

In [86]:
for genome in genomes:
    for gene in positive_genomes[genome]:
        if len(gene) >= 60:
            positive_genomes[genome].remove(gene)
    for gene in negative_genomes[genome]:
        if len(gene) >= 60:
            negative_genomes[genome].remove(gene)

In [87]:
def filter_duplicate_subdirectories(directories):
    """
    Returns a dictionary where the keys are directory names and the values are lists of subdirectories,
    filtered to exclude duplicates except for the first encounter of each subdirectory name
    in the order of directories sorted in lexicographic order.
    """
    filtered_directory_indices = create_dict(directories.keys())
    filtered_directory_genes = create_dict(directories.keys())
    subdirectory_set = set()
    for directory_name in sorted(directories.keys()):
        for i in range(len(directories[directory_name])):
            if not directories[directory_name][i] in subdirectory_set:
                filtered_directory_indices[directory_name].append(i)
                filtered_directory_genes[directory_name].append(directories[directory_name][i])
                subdirectory_set.add(directories[directory_name][i])
            else:
                continue
    return filtered_directory_genes

In [88]:
#running filtering on all genomes
positive_filtered_genes = filter_duplicate_subdirectories(positive_genomes)
negative_filtered_genes = filter_duplicate_subdirectories(negative_genomes)

In [89]:
#sanity check that there are no duplicates and no shared elements in coding regions and noncoding regions
print('There are ' + str(len(collections.Counter(flatten(positive_filtered_genes.values())) & collections.Counter(flatten(negative_filtered_genes.values())))) + ' intersected sequences')
assert(len(flatten(positive_filtered_genes.values())) == len(set(flatten(positive_filtered_genes.values()))))
assert(len(flatten(negative_filtered_genes.values())) == len(set(flatten(negative_filtered_genes.values()))))


There are 0 intersected sequences


In [90]:
#data after filtering
print('There are, in total, ' + str(len(flatten(positive_filtered_genes.values()))) + ' coding regions')
print('There are, in total, ' + str(len(flatten(negative_filtered_genes.values()))) + ' noncoding regions')


There are, in total, 128614 coding regions
There are, in total, 3367377 noncoding regions


In [91]:
#get all the indices:
positive_filtered_genes_indices = {}
negative_filtered_genes_indices = {}

def indices_of_elements(list1, list2):
    indices = []
    for element in list1:
        try:
            index = list2.index(element)
            indices.append(index)
        except ValueError:
            print('sequence not found!')
            pass
    return indices

for genome in genomes:
    positive_testing_sequences = list(pd.read_csv(coding_regions_data_dir_path + genome + '/coding_train.csv')['sequence'])
    negative_testing_sequences = list(pd.read_csv(noncoding_regions_data_dir_path + genome + '/all_noncoding_test.csv')['sequence'])
    
    filtered_positive_genes = positive_filtered_genes[genome]
    filtered_negative_genes = negative_filtered_genes[genome]
    
    positive_filtered_genes_indices[genome] = indices_of_elements(filtered_positive_genes, positive_testing_sequences)
    negative_filtered_genes_indices[genome] = indices_of_elements(filtered_negative_genes, negative_testing_sequences)
    

In [92]:
#getting train and test folders:
MAX_FOLD = 10
post_processed_data_PATH = '/Users/tonytu/Desktop/old_stuff/berkeley/fall2022/amiralilab/post_processed_data'
all_genomes = os.listdir(post_processed_data_PATH)
all_genomes.remove('.DS_Store')


def retrieve_train_test_folders(genomes, hold_out_index):
    num_genomes = len(genomes)
    if hold_out_index == MAX_FOLD - 1:
        test_split = genomes[int((num_genomes//10) * hold_out_index): ]
        train_split = list(set(genomes) - set(test_split))
        assert len(train_split) + len(test_split) == num_genomes, "numbers do not match up!"
        return train_split, test_split
    test_split = genomes[int((num_genomes//10) * hold_out_index): int(num_genomes//10 * (hold_out_index + 1))]
    train_split = list(set(genomes) - set(test_split))
    assert len(train_split) + len(test_split) == num_genomes, "numbers do not match up!"
    return train_split, test_split
    
# train_genomes, test_genomes = retrieve_train_test_folders(all_genomes, 9)


In [138]:
random.seed(5)
torch.manual_seed(5)
def load_data(genomes):
    all_x = np.array([])
    all_y = np.array([])

    random.seed(5)
    torch.manual_seed(5)
    for genome in genomes:
        
        positive_indices = np.array(positive_filtered_genes_indices[genome]) 
        negative_indices = np.array(negative_filtered_genes_indices[genome])
        if positive_indices.shape[0] != 0:
            genome_train_coding_embeddings = torch.load(coding_regions_data_dir_path + genome + '/coding_train_emb/train_mean.pt')
            genome_train_coding_labels = np.ones(genome_train_coding_embeddings.shape[0])[positive_indices]
            genome_train_coding_embeddings = genome_train_coding_embeddings[positive_indices]
            assert genome_train_coding_embeddings.shape[0] == len(positive_indices)

        if negative_indices.shape[0] != 0:
            genome_train_noncoding_embeddings = torch.load(noncoding_regions_data_dir_path+ genome + '/noncoding_test_emb/test_mean.pt')
            genome_train_noncoding_labels = np.zeros(genome_train_noncoding_embeddings.shape[0])[negative_indices]
            genome_train_noncoding_embeddings = genome_train_noncoding_embeddings[negative_indices]
            assert genome_train_noncoding_embeddings.shape[0] == len(negative_indices)

        if all_x.shape[0] == 0:
            all_x = genome_train_coding_embeddings
            all_y = genome_train_coding_labels
            all_x = np.concatenate((all_x, genome_train_noncoding_embeddings), axis=0)
            all_y = np.concatenate((all_y, genome_train_noncoding_labels), axis=0)
            continue

        if positive_indices.shape[0] != 0:
            all_x = np.concatenate((all_x, genome_train_coding_embeddings), axis=0)
            all_y = np.concatenate((all_y, genome_train_coding_labels), axis=0)
        
        if negative_indices.shape[0] != 0:
            all_x = np.concatenate((all_x, genome_train_noncoding_embeddings), axis=0)
            all_y = np.concatenate((all_y, genome_train_noncoding_labels), axis=0)
    return all_x, all_y


In [148]:
#sanity check that the dimension of the loaded data is equal to the dimension of the indices in the dictionary
assert np.vstack(all_data_folds_embs).shape[0] == len(flatten(positive_filtered_genes.values())) + len(flatten(negative_filtered_genes.values()))


In [149]:
#loading classifier:
from torch import nn
from torch.nn import functional as F

class Net(nn.Module):
  def __init__(self,input_shape):
    super(Net,self).__init__()
    #input embedding: 1280
    self.fc1 = nn.Linear(input_shape,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,1)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)
    torch.nn.init.xavier_uniform_(self.fc3.weight)
    
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.sigmoid(self.fc3(x))
    return x

class Net1(nn.Module):
  def __init__(self,input_shape):
    super(Net1,self).__init__()
    #input embedding: 1280
    self.fc1 = nn.Linear(input_shape,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,64)
    self.fc5 = nn.Linear(64,32)
    self.fc6 = nn.Linear(32,1)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)
    torch.nn.init.xavier_uniform_(self.fc3.weight)
    torch.nn.init.xavier_uniform_(self.fc4.weight)
    torch.nn.init.xavier_uniform_(self.fc5.weight)
    torch.nn.init.xavier_uniform_(self.fc6.weight)
    
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    x = torch.relu(self.fc4(x))
    x = torch.relu(self.fc5(x))
    x = torch.sigmoid(self.fc6(x))
    return x

class Net3(nn.Module):
  def __init__(self,input_shape):
    super(Net3,self).__init__()
    #input embedding: 1280
    self.fc1 = nn.Linear(input_shape,1)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    
  def forward(self,x):
    x = torch.sigmoid(self.fc1(x))
    return x

class Net4(nn.Module):
  def __init__(self,input_shape):
    super(Net4,self).__init__()
    #input embedding: 1280
    self.fc1 = nn.Linear(input_shape,128)
    self.fc2 = nn.Linear(128,256)
    self.fc3 = nn.Linear(256,512)
    self.fc4 = nn.Linear(512,256)
    self.fc5 = nn.Linear(256,128)
    self.fc6 = nn.Linear(128,64)
    self.fc7 = nn.Linear(64,32)
    self.fc8 = nn.Linear(32,1)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)
    torch.nn.init.xavier_uniform_(self.fc3.weight)
    torch.nn.init.xavier_uniform_(self.fc4.weight)
    torch.nn.init.xavier_uniform_(self.fc5.weight)
    torch.nn.init.xavier_uniform_(self.fc6.weight)
    torch.nn.init.xavier_uniform_(self.fc7.weight)
    torch.nn.init.xavier_uniform_(self.fc8.weight)
    
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    x = torch.relu(self.fc4(x))
    x = torch.relu(self.fc5(x))
    x = torch.relu(self.fc6(x))
    x = torch.relu(self.fc7(x))
    x = torch.sigmoid(self.fc8(x))
    return x

def flatten(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list

In [150]:
def create_train_test_data(all_data_folds_embs, all_data_folds_labels, fold):
    test_split = [fold]
    train_split = list(set(list(range(10))) - set([fold]))
    train_x = np.vstack([all_data_folds_embs[i] for i in train_split])
    train_y = np.concatenate([all_data_folds_labels[i] for i in train_split])
    test_x = np.vstack([all_data_folds_embs[i] for i in test_split])
    test_y = np.concatenate([all_data_folds_labels[i] for i in test_split])
    return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = create_train_test_data(all_data_folds_embs, all_data_folds_labels, 0)

#checking dimensions:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
    

(3157371, 1280)
(3157371,)
(338620, 1280)
(338620,)


In [151]:
#randomly shuffling train_x, train_y and their corresponding labels
random.seed(5)
torch.manual_seed(5)
training_indices = np.arange(train_x.shape[0])
test_indices = np.arange(test_x.shape[0])
np.random.shuffle(training_indices)
np.random.shuffle(test_indices)

In [152]:
train_x = train_x[training_indices]
train_y = train_y[training_indices]
test_x = test_x[test_indices]
test_y = test_y[test_indices]

In [153]:
random.seed(5)
torch.manual_seed(5)
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length
trainset = dataset(train_x, train_y)
#DataLoader
trainloader = DataLoader(trainset,batch_size=64,shuffle=False)

In [154]:
#hyper parameters
random.seed(5)
torch.manual_seed(5)
learning_rate = 0.01
epochs = 300
# Model , Optimizer, Loss
ecoli_model = Net4(input_shape=train_x.shape[1])
optimizer = torch.optim.SGD(ecoli_model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [ ]:
#forward loop
random.seed(5)
torch.manual_seed(5)
losses = []
accur = []
for i in range(epochs):
  for j,(x_train,y_train) in enumerate(trainloader):
    
    #calculate output
    output = ecoli_model(x_train)
 
    #calculate loss
    loss = loss_fn(output,y_train.reshape(-1,1))
 
    #accuracy
    predicted = ecoli_model(torch.tensor(x_train,dtype=torch.float32))
    acc = (predicted.reshape(-1).detach().numpy().round() == np.array(y_train)).mean()
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if i%50 == 0:
    losses.append(loss)
    accur.append(acc)
    print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))

# Training 10 fold cross validation on all coding + noncoding regions

In [ ]:
for epoch in range(0, 10, 1):
    train_x, train_y, test_x, test_y = create_train_test_data(all_data_folds_embs, all_data_folds_labels, epoch)
    
    random.seed(5)
    torch.manual_seed(5)
    training_indices = np.arange(train_x.shape[0])
    test_indices = np.arange(test_x.shape[0])
    np.random.shuffle(training_indices)
    np.random.shuffle(test_indices)
    
    train_x = train_x[training_indices]
    train_y = train_y[training_indices]
    test_x = test_x[test_indices]
    test_y = test_y[test_indices]
    
    trainset = dataset(train_x, train_y)
    #DataLoader
    trainloader = DataLoader(trainset,batch_size=64,shuffle=False)
    
    #hyper parameters
    random.seed(5)
    torch.manual_seed(5)
    learning_rate = 0.01
    epochs = 300
    # Model , Optimizer, Loss
    ecoli_model = Net1(input_shape=train_x.shape[1])
    optimizer = torch.optim.SGD(ecoli_model.parameters(),lr=learning_rate)
    loss_fn = nn.BCELoss()
    
    random.seed(5)
    torch.manual_seed(5)
    losses = []
    accur = []
    for i in range(epochs):
      for j,(x_train,y_train) in enumerate(trainloader):

        #calculate output
        output = ecoli_model(x_train)

        #calculate loss
        loss = loss_fn(output,y_train.reshape(-1,1))

        #accuracy
        predicted = ecoli_model(torch.tensor(x_train,dtype=torch.float32))
        acc = (predicted.reshape(-1).detach().numpy().round() == np.array(y_train)).mean()
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      if i%50 == 0:
        losses.append(loss)
        accur.append(acc)
        print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))
    
    #saving model
    PATH = "/Users/tonytu/desktop/old_stuff/berkeley/fall2022/amiralilab/all_available_data_final_ten_fold_cross_validation_models/six_layer_fold_" + str(epoch) + "_genomes.pt"
    torch.save(ecoli_model, PATH)
    
    

/Users/tonytu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch 0	loss : 0.02037082053720951	 accuracy : 1.0
epoch 50	loss : 0.014096093364059925	 accuracy : 1.0
epoch 100	loss : 0.010964148677885532	 accuracy : 1.0
epoch 150	loss : 0.008270354010164738	 accuracy : 1.0
epoch 200	loss : 0.0068092867732048035	 accuracy : 1.0
epoch 250	loss : 0.007393664214760065	 accuracy : 1.0
epoch 0	loss : 0.09614776819944382	 accuracy : 0.9615384615384616
epoch 50	loss : 0.11876317113637924	 accuracy : 0.9615384615384616
epoch 100	loss : 0.11171449720859528	 accuracy : 0.9615384615384616
epoch 150	loss : 0.10179932415485382	 accuracy : 0.9615384615384616
epoch 200	loss : 0.11020828038454056	 accuracy : 0.9615384615384616
epoch 250	loss : 0.08225217461585999	 accuracy : 0.9615384615384616
epoch 0	loss : 0.0177464596927166	 accuracy : 1.0
epoch 50	loss : 0.0021531996317207813	 accuracy : 1.0
epoch 100	loss : 0.0021889079362154007	 accuracy : 1.0
epoch 150	loss : 0.0014818101190030575	 accuracy : 1.0
epoch 200	loss : 0.0011308063985779881	 accuracy : 1.0
epoch